<div align="right"><i>Matías Torres Esteban<br>Diciembre, 2025</i></div>

# Generación Aumentada por Recuperación (RAG)

En un sistema de recuperación de información (SRI), dada una consulta $q$ de un usuario se recuperan de una colección de documentos $D$ los $k$ documentos más relevantes a $q$ [1]. El lenguaje en el que se expresan los documentos y la consulta, la forma en la que se representa en una computadora la colección $D$ y la definición de relevancia varían de acuerdo al dominio de aplicación y a las necesidades del usuario. Los motores de búsqueda de Google y Yahoo, por ejemplo, son SRIs donde los documentos son páginas web y las consultas se expresan en lenguaje natural.

En el contexto de la inteligencia artificial y particularmente de los agentes razonadores, podemos conceptualizar a $D$ como parte de la base de conocimiento de un sistema inteligente, y a los documentos recuperados como premisas relevantes con las que se llevará a cabo un proceso de deducción para contestar $q$. Con el surgimiento de los grandes modelos de lenguaje (LLMs) se desarrollaron nuevos modelos de razonamiento subsimbólicos y neurosimbólicos que hacen uso de SRIs en el proceso de deducción [2]. Uno de estos modelos es el de **generación aumentada por recuperación** (RAG), el cual combina LLMs con SRIs para responder preguntas que requieren de conocimiento experto [3]. ChatGPT implementa este mecanismo cuando realiza búsquedas en la web:

![Ejemplo de interacción RAG con ChatGPT](https://github.com/matizzat/InforSanLuis-2025-LLMs/blob/main/imagenes/rag_messi_chatgpt.png?raw=1)

En esa sección implementaremos un sistema RAG que combina LLMs con SRIs y bases de datos vectoriales para responder preguntas del dominio de psicología

### Preliminares

Instalamos la librería (ChromaDB)[https://docs.trychroma.com/docs/overview/getting-started], la cual nos permitirá construir una base de datos vectorial donde almacenaremos nuestros documentos de psicología. Una vez que lo instalen, Google Colab les solicitará reiniciar la sesión:

In [3]:
!pip install chromadb

Instalamaos una librería que nos permitirá visualizar nuestra base de datos vectorial y ver como están distribuidos nuestros documentos en el espacio:

In [2]:
!pip install umap-learn[plot]

Importamos recursos importantes desde Github:

In [1]:
!git clone https://github.com/matizzat/InforSanLuis-2025-LLMs
%cd InforSanLuis-2025-LLMs

fatal: destination path 'InforSanLuis-2025-LLMs' already exists and is not an empty directory.
/content/InforSanLuis-2025-LLMs


Instanciamos nuestro cliente Genai para acceder al LLM Gemini:

In [5]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
cliente_genai = genai.Client(api_key=GOOGLE_API_KEY)

### Creación y Visualización de la Base Semántica

En esta sección crearemos nuestra base de datos semántica con ayuda de la librería ChromaDB. Nuestro lote de documentos es del dominio de psicología y fueron tomador del diccionario [APA](https://dictionary.apa.org/).

Importamos librerías necesarias:

In [34]:
from sentence_transformers import SentenceTransformer
from google.colab import userdata
from google.genai import types
from functools import partial
from google import genai
import pandas as pd
import numpy as np
import csv as csv
import umap.plot
import chromadb
import pprint
import umap
import json

Importamos a un modelo **Sentence Transformer**, el cual tiene como función principal convertir un texto $w$ en un vector numérico $\vec{w}$ el cual, implícitamente, representa la semántica de $w$. Los modelos transformadores de sentencias se construyen a partir de los LLMs codificadores tales como BERT y RoBERTa:

In [32]:
sentence_transformer =  SentenceTransformer("all-MiniLM-L6-v2")

Abrimos el lote de documentos de psicología. Cada documento posee un título y un texto asociado. En total hay 107 documentos y nosotros imprimimos los primeros 5:

In [37]:
with open('./datos/psicologia.json', 'r') as f:
    chunks = json.load(f)
pprint.pprint(chunks[0:5])

[{'text': 'Mental health is a state of mind characterized by emotional '
          'well-being, good behavioral adjustment, relative freedom from '
          'anxiety and disabling symptoms, and a capacity to establish '
          'constructive relationships and cope with the ordinary demands and '
          'stresses of life.\n',
  'title': 'mental_health'},
 {'text': 'Flourishing is a condition denoting good mental and physical '
          'health: the state of being free from illness and distress but, more '
          'important, of being filled with vitality and functioning well in '
          'one’s personal and social life.\n',
  'title': 'flourishing'},
 {'text': 'Languishing is the condition of absence of mental health, '
          'characterized by ennui, apathy, listlessness, and loss of interest '
          'in life.\n',
  'title': 'languishing'},
 {'text': 'Depression is a negative affective state, ranging from unhappiness '
          'and discontent to an extreme feeling o

Convertimos cada uno de los textos de nuestro lote de datos en un vector numérico llamado **embedding** mediante el Sentence Transformer:

In [9]:
embeddings = sentence_transformer.encode([chunk['text'] for chunk in chunks])

In [10]:
cli = chromadb.PersistentClient(path="./datos/base_semantica")

In [11]:
col = cli.create_collection(
    name="ST_med2",
    embedding_function=None,
    configuration={
        "hnsw": {
            "space": "cosine",
            "ef_construction": 1000,
            "ef_search": 200
        }
    },
    metadata = {
        "description": "Embeddings del lote de documentos de psicología"
    }
)

In [12]:
U = umap.UMAP(n_components = 2, n_neighbors = 15, min_dist = 0.1, metric = 'cosine', densmap = True)
U.fit(embeddings_textos)

UMAP(angular_rp_forest=True, densmap=True, metric='cosine', tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True})

In [13]:
hover_data = pd.DataFrame({'title': [chunk['title'] for chunk in chunks],
                           'text': [chunk['text'] for chunk in chunks]})

In [14]:
umap.plot.output_notebook()

In [15]:
p = umap.plot.interactive(U, hover_data=hover_data, point_size=10)
umap.plot.show(p)

Agregamos los datos a nuestra base de datos vectorial:

In [16]:
col.add(
    ids = [chunk['title'] for chunk in chunks],
    embeddings = embeddings_textos,
    documents = textos
)

In [19]:
q = "What is dementia?"
qe = ST.encode(q)
knn_q = col.query(query_embeddings = [qe], n_results = 5)
for d in knn_q['documents'][0]:
    print(d)

Dementia praecox is a progressively deteriorating psychotic disorder marked by severe, incurable cognitive disintegration beginning in early adulthood.

Alzheimer’s disease is a progressive neurodegenerative disease characterized by cortical atrophy, neuronal death, synapse loss, and accumulation of amyloid plaques and neurofibrillary tangles, causing dementia and a significant decline in functioning.
Early features include deficits in memory (e.g., rapid forgetting of new information, impaired recall and recognition), anomia, executive dysfunction, depressive symptoms, and subtle personality changes such as decreased energy, social withdrawal, indifference, and impulsivity.
As the disease progresses, there is global deterioration of cognitive capacities with intellectual decline, aphasia, agnosia, and apraxia as well as behavioral features, including apathy, emotional blunting, mood-dependent delusions, decreased sleep and appetite, and increased motor activity (e.g., restlessness, wa

In [26]:
system_prompt = """You are an expert in the domain of psychology.
You answer the question according to the given passages with short and concise answers.
If the answer can not be derived from the passages, say that you do not have enough knowledge.
"""
user_prompt = """Passage:
@
{passage}
@
Question:
{question}
Answer:"""

In [30]:
q = "What are dinousaurs?"
qe = ST.encode(q)
knn_q = col.query(query_embeddings = [qe], n_results = 5)
top = knn_q['documents'][0][0]
formatted_user_prompt = user_prompt.format(passage = top, question = q)
print(formatted_user_prompt)
print(invocar_llm(sistema = system_prompt, usuario = formatted_user_prompt))

Passage:
@
The arachnoid mater is the middle one of the three membranous layers (meninges) covering the surface of the brain and spinal cord, so called because its strands of tissue resemble spiders’ webs.
It is also called arachnoid or arachnoid membrane.

@
Question:
What are dinousaurs?
Answer:
I do not have enough knowledge.
